In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import *
from data import prepare_data_vocab, live_feat
from torch.nn.utils.rnn import unpack_sequence
from torch_geometric.data import Data


import torch.optim as optim
from torch_geometric.data import Data

['store', 'double', 'TMP_VAR(D)', 'TMP_FUNCTION_NAME', ';', 'load', 'TMP_VAR', 'double', 'TMP_VAR(D)', ';', 'assignment', 'TMP_POINTER_MEMBER', 'TMP_FUNCTION_NAME', ';', 'assignment', 'TMP_VAR', 'TMP_FUNCTION_NAME', ';', 'math_op', 'TMP_VAR', 'TMP_NUMBER', '/', 'TMP_FUNCTION_NAME', ';', 'function_call', 'GSL_MIN_DBL', ['TMP_VAR', 'TMP_VAR'], ';', 'phi', 'TMP_VAR', 'TMP_VAR: TMP_VAR, TMP_VAR: TMP_VAR, TMP_VAR: TMP_VAR, TMP_VAR: TMP_VAR, TMP_VAR: TMP_VAR', ';']


In [77]:
word_to_idx, idx_to_word, data, data_emb = prepare_data_vocab("data", func=live_feat, function_num=0)

In [78]:
cnt = 0
for i in idx_to_word.keys():
    print(idx_to_word[i], end='\t')
    cnt+= 1

print("\n----------\n",cnt)

(INFUNCTION_VAR * INFUNCTION_VAR)TMP_FUNCTION_NAME	(INFUNCTION_VAR *)TMP_VAR	*	+	-	-TMP_NUMBER	-TMP_VAR	/	;	INFUNCTION_VAR	INFUNCTION_VAR.INFUNCTION_VAR	TMP_FUNCTION_ARG_1	TMP_FUNCTION_ARG_10	TMP_FUNCTION_ARG_11	TMP_FUNCTION_ARG_2	TMP_FUNCTION_ARG_3	TMP_FUNCTION_ARG_4	TMP_FUNCTION_ARG_5	TMP_FUNCTION_ARG_6	TMP_FUNCTION_ARG_7	TMP_FUNCTION_ARG_8	TMP_FUNCTION_ARG_9	TMP_FUNCTION_NAME	TMP_NUMBER	TMP_POINTER_MEMBER	TMP_VAR	TMP_VAR(INFUNCTION_VAR)	TMP_VAR(TMP_FUNCTION_NAME)	assignment	char	double	float	function_call	gsl_error	int	load	math_op	phi	size_t	store	
----------
 40


In [91]:
#define InputEncoder (RNN) and GNN
in_dim = 8
hidden_dim = 64
num_layers = 7
bidirectional = True
out_dim    = 128

int_dir = 2 if bidirectional else 1

bb_enc = InputEncoder(word_to_idx, 
                      in_dim=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional)
 
gnn = GNNModel(in_dim=int_dir*hidden_dim,
               out_dim=out_dim,
               layer_dims=[16, 32])

In [15]:
in_dim = 8
hidden_dim = 16
num_layers = 7
bidirectional = True
out_dim    = 32

int_dir = 2 if bidirectional else 1
integrated = IntegratedModel(word_to_idx, 
                      in_dim_lstm=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional,
                        out_dim=out_dim,
                        layer_dims=[16, 32])

In [6]:
assert len(data)==len(data_emb)
for i in range(len(data)):
    assert len(data[i][2])==len(data_emb[i][1])

#define InputEncoder (RNN) and GNN
in_dim = 8
hidden_dim = 16
num_layers = 7
bidirectional = True
out_dim    = 16

int_dir = 2 if bidirectional else 1

bb_enc = InputEncoder(word_to_idx, 
                      in_dim=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional)
 
gnn = GNNModel(in_dim=int_dir*hidden_dim,
               out_dim=out_dim,
               layer_dims=[16, 32])

In [38]:
initial_node_embeddings = []
new_data_emb = []
for graph in data_emb:
    edge_index, node_embs = graph
    #if torch.tensor(edge_index).shape[0]==0:
    if len(edge_index) != 0:
        edge_index = torch.tensor(edge_index).T
    else:
        edge_index = torch.tensor([[0],[0]])
    
    out, h, c = bb_enc(node_embs)
    node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])

    assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)

            #Step 2: gnn forward pass
    graph = Data(edge_index=edge_index,
                        x=node_feats)
    initial_node_embeddings.append(torch.rand((graph.num_nodes, out_dim), requires_grad=True))
    new_data_emb.append([edge_index, node_embs, initial_node_embeddings])

In [28]:
edge_index, node_embs = data_emb[0]
#if torch.tensor(edge_index).shape[0]==0:

if len(edge_index) != 0:
    edge_index = torch.tensor(edge_index).T
else:
    edge_index = torch.tensor([[0],[0]])
            
        
out, h, c = bb_enc(node_embs)

In [30]:
out

PackedSequence(data=tensor([[ 0.0196, -0.0244, -0.0206,  ...,  0.0122,  0.0122, -0.0505],
        [ 0.0196, -0.0243, -0.0206,  ...,  0.0122,  0.0119, -0.0505],
        [ 0.0180, -0.0271, -0.0214,  ...,  0.0167,  0.0171, -0.0428],
        ...,
        [ 0.0216, -0.0767, -0.0376,  ...,  0.0159,  0.0168, -0.0404],
        [ 0.0170, -0.0873, -0.0395,  ...,  0.0135,  0.0149, -0.0244],
        [ 0.0171, -0.0872, -0.0395,  ...,  0.0135,  0.0148, -0.0244]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([14,  6,  6,  2,  2]), sorted_indices=tensor([ 0,  6,  2,  4,  8, 10,  1,  3,  5,  7,  9, 11, 12, 13]), unsorted_indices=tensor([ 0,  6,  2,  7,  3,  8,  1,  9,  4, 10,  5, 11, 12, 13]))

In [16]:
import random
import copy
criterion = nn.MSELoss()
optimizer = optim.Adam(integrated.parameters(), lr=0.01)
torch.nn.utils.clip_grad_norm_(integrated.parameters(), max_norm=1)

epoch_num = 100
# Training loop



for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    Min = 0
    Max = 0
    Mean = 0
    for graph in data_emb:
        optimizer.zero_grad()
        j += 1
        edge_index, node_embs = graph
        #if torch.tensor(edge_index).shape[0]==0:
        if len(edge_index) != 0:
            edge_index = torch.tensor(edge_index).T
        else:
            edge_index = torch.tensor([[0],[0]])
            
        
       
        #random_indices = random.sample(range(0, 128), 8)
        random_indices = [10*x for x in range((int)(int_dir*hidden_dim/10))]        
        

        gnn_output, node_feats = integrated(node_embs, edge_index)
        
        assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)
       
        
        
        # Unsupervised loss (dummy loss for illustration)
        #loss = criterion(output, random_target[j])
        loss = criterion(gnn_output, node_feats)

        # Backward pass and optimization
        total_loss += loss.item()
        total_items += node_feats.shape[0]

        
        
        loss.backward()
        optimizer.step()

        Min += node_feats.min()/len(node_feats)
        Mean += node_feats.mean()/len(node_feats)
        Max += node_feats.max()/len(node_feats)
        
        if epoch%2==0 or epoch==epoch_num-1:
            avg_loss = total_loss / (j+1)
            print(f"Min: {Min:.10f}, Mean: {Mean:.10f}, Max: {Max:.10f}", end=' |#| ')
            print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.10f}', end='\r')

        #total_loss += unsupervised_loss.item()

    # Print or log the average loss for monitoring
    if epoch%2==0 or epoch==epoch_num-1:
        print()


RuntimeError: index 11 is out of bounds for dimension 0 with size 11

In [123]:
node_embs

[tensor([1, 1, 1, 2, 3]),
 tensor([1]),
 tensor([6, 1, 3]),
 tensor([1]),
 tensor([6, 2, 3]),
 tensor([1]),
 tensor([6, 0, 0, 0, 2]),
 tensor([1]),
 tensor([6, 2, 3]),
 tensor([1]),
 tensor([6, 2, 3]),
 tensor([1]),
 tensor([6]),
 tensor([4])]

In [128]:
min = 10000
max = 0
for graph in data_emb:
    edge_index, node_embs = graph
    if len(edge_index) != 0:
        if len(node_embs)>max:
            max = len(node_embs)
        if len(node_embs)<min:
            min = len(node_embs)
print("Max: ", max)
print("Min: ", min)

Max:  128
Min:  2


In [99]:
criterion = nn.MSELoss()
optimizer = optim.Adam(gnn.parameters(), lr=0.01)

epoch_num = 300
# Training loop

random_target = torch.rand((len(data_emb), 1, out_dim))

for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for graph in data_emb:
        j += 1
        edge_index, node_embs = graph
        #if torch.tensor(edge_index).shape[0]==0:
        if len(edge_index) != 0:
            edge_index = torch.tensor(edge_index).T
        else:
            edge_index = torch.tensor([[0],[0]])
            
        
        out, h, c = bb_enc(node_embs)
        node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])

        assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)

                #Step 2: gnn forward pass
        graph = Data(edge_index=edge_index,
                            x=node_feats)
        
        output = gnn(graph)
        # Unsupervised loss (dummy loss for illustration)
        loss = criterion(output, random_target[j])
        #loss = criterion(output, initial_node_embeddings[j])

        # Backward pass and optimization
        total_loss  += loss.item()
        total_items += graph.x.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch%10==0 or epoch==epoch_num-1:
            avg_loss = total_loss / len(data_emb)
            print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')

        #total_loss += unsupervised_loss.item()

    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        print()


For 578 functions: Epoch 1/300, Average Loss: 0.08449


KeyboardInterrupt: 

In [115]:
node_feats

tensor([[-1.6942e-06,  3.6233e-06, -7.3788e-04, -4.4494e-05,  3.6724e-08,
         -7.6004e-01, -2.7473e-03,  5.2556e-04,  3.6020e-06, -4.5144e-05,
         -9.2002e-07,  6.8339e-02, -2.3965e-04,  1.6933e-03, -1.9910e-04,
         -2.3565e-04,  4.8033e-02,  9.4108e-02, -7.9327e-02,  1.0815e-01,
          1.6961e-02, -1.9245e-02, -3.6383e-03,  1.5153e-01, -4.3944e-02,
         -2.2803e-02, -7.2129e-02,  7.6061e-02, -3.1655e-02, -1.4341e-01,
          9.2256e-02, -1.8506e-01],
        [-1.6942e-06,  3.6233e-06, -7.3788e-04, -4.4494e-05,  3.6724e-08,
         -7.6004e-01, -2.7473e-03,  5.2556e-04,  3.6020e-06, -4.5144e-05,
         -9.2002e-07,  6.8339e-02, -2.3965e-04,  1.6933e-03, -1.9910e-04,
         -2.3565e-04,  4.8033e-02,  9.4108e-02, -7.9327e-02,  1.0815e-01,
          1.6961e-02, -1.9245e-02, -3.6383e-03,  1.5153e-01, -4.3944e-02,
         -2.2803e-02, -7.2129e-02,  7.6061e-02, -3.1655e-02, -1.4341e-01,
          9.2256e-02, -1.8506e-01],
        [-1.6942e-06,  3.6233e-06, -7.37

In [98]:
for name, param in bb_enc.named_parameters():
    if param.grad is not None:
        print(f"{name} grad norm: {param.grad.norm()}")
    else:
        print(f"{name} grad is None")

emb.weight grad norm: 4.2484352889005095e-05
lstm.weight_ih_l0 grad norm: 0.0003254949115216732
lstm.weight_hh_l0 grad norm: 7.605506834806874e-05
lstm.bias_ih_l0 grad norm: 0.0003012344823218882
lstm.bias_hh_l0 grad norm: 0.0003012344823218882
lstm.weight_ih_l0_reverse grad norm: 0.00032709658262319863
lstm.weight_hh_l0_reverse grad norm: 7.603189442306757e-05
lstm.bias_ih_l0_reverse grad norm: 0.000313809810904786
lstm.bias_hh_l0_reverse grad norm: 0.000313809810904786
lstm.weight_ih_l1 grad norm: 0.000558680621907115
lstm.weight_hh_l1 grad norm: 0.00022624272969551384
lstm.bias_ih_l1 grad norm: 0.0010487716645002365
lstm.bias_hh_l1 grad norm: 0.0010487715480849147
lstm.weight_ih_l1_reverse grad norm: 0.0004790965758729726
lstm.weight_hh_l1_reverse grad norm: 0.00016279509873129427
lstm.bias_ih_l1_reverse grad norm: 0.0009165560477413237
lstm.bias_hh_l1_reverse grad norm: 0.0009165561059489846
lstm.weight_ih_l2 grad norm: 0.0014243311015889049
lstm.weight_hh_l2 grad norm: 0.000649143

In [100]:
for name, param in bb_enc.named_parameters():
    if param.grad is not None:
        print(f"{name} grad norm: {param.grad.norm()}")
    else:
        print(f"{name} grad is None")

emb.weight grad norm: 4.2484352889005095e-05
lstm.weight_ih_l0 grad norm: 0.0003254949115216732
lstm.weight_hh_l0 grad norm: 7.605506834806874e-05
lstm.bias_ih_l0 grad norm: 0.0003012344823218882
lstm.bias_hh_l0 grad norm: 0.0003012344823218882
lstm.weight_ih_l0_reverse grad norm: 0.00032709658262319863
lstm.weight_hh_l0_reverse grad norm: 7.603189442306757e-05
lstm.bias_ih_l0_reverse grad norm: 0.000313809810904786
lstm.bias_hh_l0_reverse grad norm: 0.000313809810904786
lstm.weight_ih_l1 grad norm: 0.000558680621907115
lstm.weight_hh_l1 grad norm: 0.00022624272969551384
lstm.bias_ih_l1 grad norm: 0.0010487716645002365
lstm.bias_hh_l1 grad norm: 0.0010487715480849147
lstm.weight_ih_l1_reverse grad norm: 0.0004790965758729726
lstm.weight_hh_l1_reverse grad norm: 0.00016279509873129427
lstm.bias_ih_l1_reverse grad norm: 0.0009165560477413237
lstm.bias_hh_l1_reverse grad norm: 0.0009165561059489846
lstm.weight_ih_l2 grad norm: 0.0014243311015889049
lstm.weight_hh_l2 grad norm: 0.000649143

In [51]:
for name, param in integrated.named_parameters():
    if not param.requires_grad:
        print("not", param)
    

AttributeError: 'generator' object has no attribute 'grad'

In [49]:
import re

def extract_a_op_b(input_string):
    

    # Find and replace expressions within parentheses with a placeholder
    placeholder = '__PLACEHOLDER__'
    flag_par = 0
    while '(' in input_string and ')' in input_string:
        flag_par = 1
        # Find the innermost expression within parentheses
        inner_expr = re.search(r'\([^)]*\)', input_string).group(0)
        
        # Replace the inner expression with a placeholder
        input_string = input_string.replace(inner_expr, placeholder)

    # Define the original pattern
    original_pattern = r"([-+]?\b\d+(\.\d*)?(e[+-]?\d+)?\b|\b\w+\b|\[\s*\d+(\s*,\s*\d+)*\s*\])\s*([+\-*/])\s*([-+]?\b\d+(\.\d*)?(e[+-]?\d+)?\b|\b\w+\b|\[\s*\d+(\s*,\s*\d+)*\s*\])"

    # Find the first match in the modified input string
    match = re.search(original_pattern, input_string)

    if match:
        # Extract a, op, b from the match
        a = match.group(1)
        op = match.group(5)
        b = match.group(6)

        # Replace the placeholder back with the original expression
        if flag_par:
            a = a.replace(placeholder, inner_expr)
            b = b.replace(placeholder, inner_expr)

        return a, op, b
    else:
        return None

# Example usage
input_string = "_1 * 5.0e-1"
result = extract_a_op_b(input_string)

if result:
    a, op, b = result
    print("a:", a)
    print("op:", op)
    print("b:", b)
else:
    print("No match found.")


a: _1
op: *
b: 5.0e-1


In [35]:
import numpy as np
vocab = np.array(['a', 'b', 'c', 'a'])
vocab = np.unique(vocab)
word_to_idx = dict(zip(vocab, np.arange(len(vocab))))
idx_to_word = dict(zip(np.arange(len(vocab)), vocab))

In [36]:
word_to_idx

{'a': 0, 'b': 1, 'c': 2}